In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c dogs-vs-cats

In [ ]:
!ls

In [ ]:
from zipfile import ZipFile
dataset = "/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/dogs-vs-cats.zip"

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("The datset is extracted")

In [ ]:
!ls

In [ ]:
from zipfile import ZipFile
dataset = "/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/train.zip"

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print("The datset is extracted")

Checking the number of rows in the train folder

In [ ]:
import os

path, dir, files = next(os.walk("/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/train.zip"))
file_count = len(files)
print(file_count)

Printing the Name of the images

In [ ]:
file_names = os.listdir("/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/train")
print(file_names)

### Importing the Dependecies

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

#### Trying to displaying the Dag/Cat images.

Displaying Dog images

In [ ]:
img = mpimg.imread("/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/train/dog.3221.jpg")  # Update the path accordingly
plt.imshow(img)
plt.show()

In [ ]:
img.shape

Displaying Cat images

In [ ]:
img_cat = mpimg.imread("/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/train/cat.332.jpg")
plt.imshow(img)
plt.show()

In [ ]:
img_cat.shape

As we can see, none of the pictures are of same size. To train a deep learning model it is really necessary to put all the images are of same dimenstion. If not, there will be errors in batch processing, the training might fail. If somehow, you convinced using dynamic resising internally, in that case, the learned features could become inconsistent

##### Resizing the Images

In [ ]:
file_names = os.listdir("/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/train")

dogs_count = 0
cats_count = 0

for img_file in file_names:
  var_name = img_file[0:3]
  if var_name == 'dog':
    dogs_count +=1
  else:
    cats_count +=1

print("Number of Dogs:",dogs_count)
print("Number of Cats:",cats_count)


Insted of reshaping all the images, I'll take few of both the labels and put them in a diff folder with of same size.

In [ ]:
os.mkdir("/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/resized_images")


In [ ]:
originial_folder = "/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/train/"
resized_folder = "/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/resized_images/"

# taking 10000 images

for x in range(12500):
  file_name = os.listdir(originial_folder)[x] #takes 1st element
  img_path = originial_folder + file_name #concats it's with the path

  img = Image.open(img_path)
  img = img.resize((224,224)) #224,224 is because, MobileNet expects to be in this size for transfer learning.
  img = img.convert('RGB')

  new_img_path = (resized_folder + file_name)
  img.save(new_img_path)



In [ ]:
img = mpimg.imread("/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/resized_images/cat.1.jpg")
imgplt = plt.imshow(img)
plt.show()
print(img.shape)

In [ ]:
img = mpimg.imread("/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/resized_images/dog.264.jpg")
imgplt = plt.imshow(img)
plt.show()
print(img.shape)

Now that all the images are of same size, we can now move on to **Creating labels**

- Dog - 1
- cat - 0

In [ ]:
file_names = os.listdir("/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/resized_images/")
labels = []

for x in range(12500):

  # Iterating through all the names,
  # If the first 3 elements sums up to "dog", then it's gonna append 1 to the list 'labels',
  # If its a 'cat', then 0 is appended to the list.

  file_name = file_names[x]
  var_name = file_name[0:3]

  if var_name == 'dog':
    labels.append(1)
  else:
    labels.append(0)



In [ ]:
print(labels[0:10])
print(len(labels))

In [ ]:
# Let's check how many cats and dogs are there.

print(labels.count(1))
print(labels.count(0))

Before training the model, we need to convert those images into numpy arrays.

In [ ]:
import cv2
import glob

In [ ]:
image_dir = ("/Users/pavansaipendry/Desktop/Master's/Sem 1/Deep Learning/Dog v:s Cat classification using MobileNet/resized_images/")
image_extension = ['png', 'jpg']

files = []

[files.extend(glob.glob(image_dir + "*." + e)) for e in image_extension]

dog_cat_images = np.asarray([cv2.imread(file) for file in files])


In [ ]:
dog_cat_images

In [ ]:
# type of images
type(dog_cat_images)

In [ ]:
# shape
dog_cat_images.shape

### Train Test Split

In [ ]:
X = dog_cat_images
y = np.asarray(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Before train the NN, we have to scale the data.

In [ ]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [ ]:
print(X_train_scaled)

#### Building a Neural Network

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
# pip install tensorflow-hub

In [ ]:
mobilenet_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

pretrained_model  = hub.KerasLayer(mobilenet_url , input_shape = (224,224,3), trainable=False)

In [ ]:
model = tf.keras.Sequential([
    pretrained_model, 
    tf.keras.layers.Dense(2) #2 is number of classes
])

model.summary()

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['acc']
)

Model training

In [ ]:
model.fit(X_train_scaled, y_train, epochs= 5)